# Part1 - Initial Trials

In [1]:
#import the libraries
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pandas as pd
import numpy as np
import random

#set seed
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

#enable mixed precision
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')


#in case of a deletion of a directory
#import shutil
#shutil.rmtree("/content/TensorboardLogs")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#define data generator
datagen = ImageDataGenerator(
                             validation_split = 0.2)

#create datasets
train_dataset = datagen.flow_from_directory("/kaggle/input/animals10/raw-img",
                                           subset = 'training',
                                           seed = SEED,
                                           target_size = (150,150),
                                           class_mode = 'categorical')

validation_dataset = datagen.flow_from_directory("/kaggle/input/animals10/raw-img",
                                           subset = 'validation',
                                           seed = SEED,
                                           target_size = (150,150),
                                           class_mode = 'categorical')

Found 20947 images belonging to 10 classes.
Found 5232 images belonging to 10 classes.


In [3]:
def define_callbacks(model):
  es = tf.keras.callbacks.EarlyStopping(patience = 3,verbose = 1, restore_best_weights = True)
  mc = tf.keras.callbacks.ModelCheckpoint(filepath = f"./ModelCheckpoints/{model.name}.ckpt",
                                         save_best_only = True,
                                         save_weights_only = True)
  tb = tf.keras.callbacks.TensorBoard(log_dir = f"./TensorboardLogs/{model.name}")
  return es,mc,tb

In [5]:
#create a model dictionary
models = {}
histories = {}

#set up global variables
MODEL_NAME = 'model1'
HISTORY_NAME = 'history1'
FILTERS = 32
ACTIVATION = 'relu'

#create the model
inputs = tf.keras.Input(shape = (150,150,3))
x = layers.Rescaling(1/255.)(inputs)

x = tf.keras.layers.Conv2D(filters = FILTERS,kernel_size = 3, activation = ACTIVATION)(x)
x = layers.MaxPool2D()(x)

x = tf.keras.layers.Conv2D(filters = FILTERS,kernel_size = 3, activation = ACTIVATION)(x)
x = layers.MaxPool2D()(x)

x = tf.keras.layers.Conv2D(filters = FILTERS,kernel_size = 3, activation = ACTIVATION)(x)
x = layers.MaxPool2D()(x)

x = layers.Flatten()(x)
outputs = layers.Dense(10,activation = 'softmax')(x)

models[MODEL_NAME] = tf.keras.Model(inputs,outputs,name = MODEL_NAME)

#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

  
#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(
    train_dataset,
    validation_data = validation_dataset,
    epochs = 5,
    batch_size = 8,
)

Epoch 1/5
655/655 [==============================] - 187s 268ms/step - loss: 1.6548 - accuracy: 0.4308 - val_loss: 1.3487 - val_accuracy: 0.5281
Epoch 2/5
655/655 [==============================] - 51s 77ms/step - loss: 1.2147 - accuracy: 0.5938 - val_loss: 1.1766 - val_accuracy: 0.6051
Epoch 3/5
655/655 [==============================] - 51s 78ms/step - loss: 0.9738 - accuracy: 0.6768 - val_loss: 1.1811 - val_accuracy: 0.6139
Epoch 4/5
655/655 [==============================] - 52s 79ms/step - loss: 0.7927 - accuracy: 0.7371 - val_loss: 1.0754 - val_accuracy: 0.6519
Epoch 5/5
655/655 [==============================] - 52s 80ms/step - loss: 0.6422 - accuracy: 0.7889 - val_loss: 1.1665 - val_accuracy: 0.6437


In [6]:
#set up global variables
MODEL_NAME = 'model2'
HISTORY_NAME = 'history2'
FILTERS = 32
ACTIVATION = 'relu'

#create the model
inputs = tf.keras.Input(shape = (150,150,3))
x = layers.Rescaling(1/255.)(inputs)

x = tf.keras.layers.Conv2D(filters = FILTERS,kernel_size = 3, activation = 'relu')(x)
x = layers.MaxPool2D()(x)

x = tf.keras.layers.Conv2D(filters = FILTERS,kernel_size = 3, activation = 'relu')(x)
x = layers.MaxPool2D()(x)

x = tf.keras.layers.Conv2D(filters = FILTERS,kernel_size = 3, activation = 'relu')(x)
#x = layers.MaxPool2D()(x)

x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(10,activation = 'softmax')(x)

models[MODEL_NAME] = tf.keras.Model(inputs,outputs,name = MODEL_NAME)

#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

  
#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(
    train_dataset,
    validation_data = validation_dataset,
    epochs = 5,
    batch_size = 8,
)

Epoch 1/5
655/655 [==============================] - 52s 77ms/step - loss: 2.1270 - accuracy: 0.2286 - val_loss: 2.0655 - val_accuracy: 0.2424
Epoch 2/5
655/655 [==============================] - 55s 84ms/step - loss: 1.9610 - accuracy: 0.2995 - val_loss: 1.9174 - val_accuracy: 0.3234
Epoch 3/5
655/655 [==============================] - 52s 80ms/step - loss: 1.8352 - accuracy: 0.3533 - val_loss: 1.7679 - val_accuracy: 0.3765
Epoch 4/5
655/655 [==============================] - 51s 78ms/step - loss: 1.7338 - accuracy: 0.3905 - val_loss: 1.7150 - val_accuracy: 0.3859
Epoch 5/5
655/655 [==============================] - 53s 81ms/step - loss: 1.6748 - accuracy: 0.4160 - val_loss: 1.6053 - val_accuracy: 0.4302


Select `Flatten`. Change the activation. (from `relu` to `swish`)


In [7]:
#set up global variables
MODEL_NAME = 'model3'
HISTORY_NAME = 'history3'
ACTIVATION = 'swish'
FILTERS = 32

#create the model
inputs = tf.keras.Input(shape = (150,150,3))
x = layers.Rescaling(1/255.)(inputs)

x = tf.keras.layers.Conv2D(filters = FILTERS,kernel_size = 3, activation = ACTIVATION)(x)
x = layers.MaxPool2D()(x)

x = tf.keras.layers.Conv2D(filters = FILTERS,kernel_size = 3, activation = ACTIVATION)(x)
x = layers.MaxPool2D()(x)

x = tf.keras.layers.Conv2D(filters = FILTERS,kernel_size = 3, activation = ACTIVATION)(x)
x = layers.MaxPool2D()(x)

x = layers.Flatten()(x)
outputs = layers.Dense(10,activation = 'softmax')(x)

models[MODEL_NAME] = tf.keras.Model(inputs,outputs,name = MODEL_NAME)

#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'],
   
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(
    train_dataset,
    validation_data = validation_dataset,
    batch_size = 8,
    epochs = 5)

Epoch 1/5
655/655 [==============================] - 56s 83ms/step - loss: 1.7532 - accuracy: 0.3940 - val_loss: 1.5704 - val_accuracy: 0.4637
Epoch 2/5
655/655 [==============================] - 53s 82ms/step - loss: 1.3991 - accuracy: 0.5338 - val_loss: 1.2919 - val_accuracy: 0.5619
Epoch 3/5
655/655 [==============================] - 50s 77ms/step - loss: 1.1782 - accuracy: 0.6108 - val_loss: 1.2842 - val_accuracy: 0.5786
Epoch 4/5
655/655 [==============================] - 51s 79ms/step - loss: 0.9961 - accuracy: 0.6724 - val_loss: 1.2131 - val_accuracy: 0.6082
Epoch 5/5
655/655 [==============================] - 53s 81ms/step - loss: 0.8304 - accuracy: 0.7278 - val_loss: 1.2097 - val_accuracy: 0.6189


Adopt `swish` activation. Increase the number of filters.

In [8]:
#set up global variables
MODEL_NAME = 'model4'
HISTORY_NAME = 'history4'
ACTIVATION = 'swish'
FILTERS = 64

#create the model
inputs = tf.keras.Input(shape = (150,150,3))
x = layers.Rescaling(1/255.)(inputs)

x = tf.keras.layers.Conv2D(filters = FILTERS,kernel_size = 3, activation = ACTIVATION)(x)
x = layers.MaxPool2D()(x)

x = tf.keras.layers.Conv2D(filters = FILTERS,kernel_size = 3, activation = ACTIVATION)(x)
x = layers.MaxPool2D()(x)

x = tf.keras.layers.Conv2D(filters = FILTERS,kernel_size = 3, activation = ACTIVATION)(x)
x = layers.MaxPool2D()(x)

x = layers.Flatten()(x)
outputs = layers.Dense(10,activation = 'softmax')(x)

models[MODEL_NAME] = tf.keras.Model(inputs,outputs,name = MODEL_NAME)

#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'],
   
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(
    train_dataset,
    validation_data = validation_dataset,
    batch_size = 8,
    epochs = 5)

Epoch 1/5
655/655 [==============================] - 59s 86ms/step - loss: 1.7214 - accuracy: 0.4075 - val_loss: 1.4449 - val_accuracy: 0.5143
Epoch 2/5
655/655 [==============================] - 56s 85ms/step - loss: 1.3004 - accuracy: 0.5697 - val_loss: 1.3441 - val_accuracy: 0.5514
Epoch 3/5
655/655 [==============================] - 56s 86ms/step - loss: 1.0275 - accuracy: 0.6629 - val_loss: 1.2329 - val_accuracy: 0.5929
Epoch 4/5
655/655 [==============================] - 57s 87ms/step - loss: 0.7658 - accuracy: 0.7483 - val_loss: 1.2881 - val_accuracy: 0.6067
Epoch 5/5
655/655 [==============================] - 56s 86ms/step - loss: 0.5246 - accuracy: 0.8259 - val_loss: 1.4849 - val_accuracy: 0.6023


Increasing filter size only overfits the model. Consider using data augmentation.

# Part 2 - Data Augmentation

In [1]:
#import the libraries
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pandas as pd
import numpy as np
import random

#set seed
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

#enable mixed precision
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')


#in case of a deletion of a directory
#import shutil
#shutil.rmtree("/content/TensorboardLogs")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def define_callbacks(model):
  es = tf.keras.callbacks.EarlyStopping(patience = 3,verbose = 1, restore_best_weights = True)
  mc = tf.keras.callbacks.ModelCheckpoint(filepath = f"./ModelCheckpoints/{model.name}.ckpt",
                                         save_best_only = True,
                                         save_weights_only = True)
 
  return es,mc

models = {}
histories = {}

In [3]:
#define data augmentation generator
train_datagen = ImageDataGenerator(rotation_range = 0.2,
                              horizontal_flip=True,
                             width_shift_range = 0.2,
                             height_shift_range = 0.2,
                             zoom_range = 0.2,
                             validation_split = 0.2)

#define a plain generator for validation data
validation_datagen = ImageDataGenerator(validation_split = 0.2)

#create datasets
train_dataset_augmented = train_datagen.flow_from_directory("/kaggle/input/animals10/raw-img",
                                           subset = 'training',
                                           seed = SEED,
                                           target_size = (150,150),
                                           class_mode = 'categorical')

validation_dataset_augmented = validation_datagen.flow_from_directory("/kaggle/input/animals10/raw-img",
                                           subset = 'validation',
                                           seed = SEED,
                                           target_size = (150,150),
                                           class_mode = 'categorical')

Found 20947 images belonging to 10 classes.
Found 5232 images belonging to 10 classes.


In [4]:
#set up global variables
MODEL_NAME = 'model5'
HISTORY_NAME = 'history5'
ACTIVATION = 'swish'
FILTERS = 64

#create the same model (model4)
inputs = tf.keras.Input(shape = (150,150,3))
x = layers.Rescaling(1/255.)(inputs)

x = tf.keras.layers.Conv2D(filters = FILTERS,kernel_size = 3, activation = ACTIVATION)(x)
x = layers.MaxPool2D()(x)

x = tf.keras.layers.Conv2D(filters = FILTERS,kernel_size = 3, activation = ACTIVATION)(x)
x = layers.MaxPool2D()(x)

x = tf.keras.layers.Conv2D(filters = FILTERS,kernel_size = 3, activation = ACTIVATION)(x)
x = layers.MaxPool2D()(x)

x = layers.Flatten()(x)
outputs = layers.Dense(10,activation = 'softmax')(x)

models[MODEL_NAME] = tf.keras.Model(inputs,outputs,name = MODEL_NAME)

#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'],
   
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(
    train_dataset_augmented,
    validation_data = validation_dataset_augmented,
    batch_size = 8,
    epochs = 5)

Epoch 1/5
655/655 [==============================] - 277s 404ms/step - loss: 1.9729 - accuracy: 0.3007 - val_loss: 1.8264 - val_accuracy: 0.3622
Epoch 2/5
655/655 [==============================] - 169s 257ms/step - loss: 1.7525 - accuracy: 0.3946 - val_loss: 1.7231 - val_accuracy: 0.4197
Epoch 3/5
655/655 [==============================] - 155s 236ms/step - loss: 1.6161 - accuracy: 0.4528 - val_loss: 1.5051 - val_accuracy: 0.4780
Epoch 4/5
655/655 [==============================] - 165s 252ms/step - loss: 1.5161 - accuracy: 0.4850 - val_loss: 1.4247 - val_accuracy: 0.5268
Epoch 5/5
655/655 [==============================] - 164s 250ms/step - loss: 1.4630 - accuracy: 0.5068 - val_loss: 1.3010 - val_accuracy: 0.5694


Now since the model did not overfitted, we can train it for longer.

In [5]:
#set up global variables
MODEL_NAME = 'model6'
HISTORY_NAME = 'history6'
ACTIVATION = 'swish'
FILTERS = 64

inputs = tf.keras.Input(shape = (150,150,3))
x = layers.Rescaling(1/255.)(inputs)

x = tf.keras.layers.Conv2D(filters = FILTERS,kernel_size = 3, activation = ACTIVATION)(x)
x = layers.MaxPool2D()(x)

x = tf.keras.layers.Conv2D(filters = FILTERS,kernel_size = 3, activation = ACTIVATION)(x)
x = layers.MaxPool2D()(x)

x = tf.keras.layers.Conv2D(filters = FILTERS,kernel_size = 3, activation = ACTIVATION)(x)
x = layers.MaxPool2D()(x)

x = layers.Flatten()(x)
outputs = layers.Dense(10,activation = 'softmax')(x)

models[MODEL_NAME] = tf.keras.Model(inputs,outputs,name = MODEL_NAME)

#define callbacks
es,mc = define_callbacks(models[MODEL_NAME])

#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'],
   
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(
    train_dataset_augmented,
    validation_data = validation_dataset_augmented,
    batch_size = 8,
    epochs = 100,
    callbacks = [es,mc])

Epoch 1/100
655/655 [==============================] - 167s 251ms/step - loss: 1.9793 - accuracy: 0.2978 - val_loss: 1.7200 - val_accuracy: 0.4060
Epoch 2/100
655/655 [==============================] - 165s 253ms/step - loss: 1.7518 - accuracy: 0.3928 - val_loss: 1.5859 - val_accuracy: 0.4537
Epoch 3/100
655/655 [==============================] - 163s 249ms/step - loss: 1.6181 - accuracy: 0.4427 - val_loss: 1.5449 - val_accuracy: 0.4820
Epoch 4/100
655/655 [==============================] - 163s 249ms/step - loss: 1.5233 - accuracy: 0.4826 - val_loss: 1.5456 - val_accuracy: 0.4962
Epoch 5/100
655/655 [==============================] - 162s 247ms/step - loss: 1.4633 - accuracy: 0.5036 - val_loss: 1.3661 - val_accuracy: 0.5443
Epoch 6/100
655/655 [==============================] - 165s 251ms/step - loss: 1.4309 - accuracy: 0.5132 - val_loss: 1.4077 - val_accuracy: 0.5334
Epoch 7/100
655/655 [==============================] - 163s 248ms/step - loss: 1.3982 - accuracy: 0.5291 - val_loss: 1

So we are still very far from having a good model. Let's change the root and try a transfer learning model.

# Part3 - Feature Extraction

In [1]:
#import the libraries
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pandas as pd
import numpy as np
import random

#set seed
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

#enable mixed precision
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')


#in case of a deletion of a directory
#import shutil
#shutil.rmtree("/content/TensorboardLogs")


def define_callbacks(model):
  es = tf.keras.callbacks.EarlyStopping(patience = 3,verbose = 1, restore_best_weights = True)
  mc = tf.keras.callbacks.ModelCheckpoint(filepath = f"./ModelCheckpoints/{model.name}.ckpt",
                                         save_best_only = True,
                                         save_weights_only = True)
  tb = tf.keras.callbacks.TensorBoard(log_dir = f"./TensorboardLogs/{model.name}")
  return es,mc

models = {}
histories = {}

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#define data augmentation generator
train_datagen = ImageDataGenerator(rotation_range = 0.2,
                              horizontal_flip=True,
                             width_shift_range = 0.2,
                             height_shift_range = 0.2,
                             zoom_range = 0.2,
                             validation_split = 0.2)

#define a plain generator for validation data
validation_datagen = ImageDataGenerator(validation_split = 0.2)

#create datasets
train_dataset_augmented = train_datagen.flow_from_directory("/kaggle/input/animals10/raw-img",
                                           subset = 'training',
                                           seed = SEED,
                                           target_size = (224,224), #EfficientNet expects 224,224 shaped images
                                           class_mode = 'categorical')

validation_dataset_augmented = validation_datagen.flow_from_directory("/kaggle/input/animals10/raw-img",
                                           subset = 'validation',
                                           seed = SEED,
                                           target_size = (224,224),
                                           class_mode = 'categorical')

Found 20947 images belonging to 10 classes.
Found 5232 images belonging to 10 classes.


In [3]:
#import a feature extraction model - freeze it
feature_extraction_model = tf.keras.applications.EfficientNetB0(include_top = False)
feature_extraction_model.trainable = False

#set up the variables
MODEL_NAME = 'model7'
HISTORY_NAME = 'history7'

#create the model
inputs = tf.keras.Input(shape = (224,224,3))
x = feature_extraction_model(inputs,training = False) #to prevent batchnorm to learn
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(10,activation = 'softmax')(x)

models[MODEL_NAME] = tf.keras.Model(inputs,outputs, name = MODEL_NAME)

#define callbacks
es,mc = define_callbacks(models[MODEL_NAME])

#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'],
   
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(
    train_dataset_augmented,
    validation_data = validation_dataset_augmented,
    batch_size = 8,
    epochs = 100,
    callbacks = [es,mc])

16705208/16705208 [==============================] - 0s 0us/step
Epoch 1/100
655/655 [==============================] - 439s 651ms/step - loss: 0.2465 - accuracy: 0.9422 - val_loss: 0.0934 - val_accuracy: 0.9765
Epoch 2/100
655/655 [==============================] - 268s 409ms/step - loss: 0.1089 - accuracy: 0.9694 - val_loss: 0.0796 - val_accuracy: 0.9782
Epoch 3/100
655/655 [==============================] - 271s 414ms/step - loss: 0.0913 - accuracy: 0.9736 - val_loss: 0.0779 - val_accuracy: 0.9776
Epoch 4/100
655/655 [==============================] - 260s 397ms/step - loss: 0.0827 - accuracy: 0.9758 - val_loss: 0.0729 - val_accuracy: 0.9784
Epoch 5/100
655/655 [==============================] - 262s 399ms/step - loss: 0.0750 - accuracy: 0.9781 - val_loss: 0.0719 - val_accuracy: 0.9784
Epoch 6/100
655/655 [==============================] - 259s 396ms/step - loss: 0.0708 - accuracy: 0.9788 - val_loss: 0.0726 - val_accuracy: 0.9795
Epoch 7/100
655/655 [==============================] 

# Conclusion

In this work, we have tried different part of neural networks for this particular image classification task. Any of the simpler models coudn't give satisfactory results. On the other hand, EfficientNetB0 (the simplest one) could successfully achive high accuracies.